In [32]:
import pandas as pd
import numpy as np
np.random.seed(1)

In [33]:
paris_listing = pd.read_csv('paris_airbnb.csv')
paris_listing['price'] = paris_listing['price'].apply(lambda x : x.replace('$','').replace(',','')).astype(float)
paris_listing = paris_listing.iloc[np.random.permutation(len(paris_listing))]

In [34]:
paris_listing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8000 entries, 4740 to 5157
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   host_response_rate    5000 non-null   object 
 1   host_acceptance_rate  0 non-null      float64
 2   host_listings_count   7999 non-null   float64
 3   latitude              8000 non-null   float64
 4   longitude             8000 non-null   float64
 5   city                  7997 non-null   object 
 6   zipcode               7930 non-null   object 
 7   state                 7977 non-null   object 
 8   accommodates          8000 non-null   int64  
 9   room_type             8000 non-null   object 
 10  bedrooms              7976 non-null   float64
 11  bathrooms             7942 non-null   float64
 12  beds                  7986 non-null   float64
 13  price                 8000 non-null   float64
 14  cleaning_fee          6250 non-null   object 
 15  security_deposit      6

In [35]:
paris_listing.head()

,host_response_rate,host_acceptance_rate,host_listings_count,latitude,longitude,city,zipcode,state,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews
4740,NaN,NaN,1.0,48.88285,2.33852,Paris,75018,Île-de-France,4,Entire home/apt,1.0,1.0,2.0,65.0,$30.00,$200.00,2,1125,6
5606,NaN,NaN,1.0,48.86006,2.34768,Paris,75001,Île-de-France,3,Entire home/apt,2.0,1.5,2.0,98.0,NaN,NaN,5,1124,1
4824,100%,NaN,3.0,48.88794,2.34532,Paris,75018,Île-de-France,2,Entire home/apt,0.0,1.0,1.0,65.0,$30.00,"$1,000.00",10,1125,0
4205,60%,NaN,1.0,48.88868,2.34111,Paris,75018,Île-de-France,2,Private room,1.0,1.0,1.0,45.0,$30.00,$200.00,5,1125,83
3228,90%,NaN,1.0,48.88686,2.33670,Paris,75018,Île-de-France,3,Entire home/apt,1.0,1.0,2.0,65.0,NaN,$200.00,5,365,5


# Suppression des donnée non pertinente
## données non numérique:
    - room_type
    - city
    - zipcode
    - state
## valeurs numérique non ordinales
    - longitute
    - lattitude

## valeurs ne décrivants pas le logement
    - host_response_rate
    - host_acceptance_rate
    - host_listings_count

In [36]:
paris_listing = paris_listing.drop(['room_type', 'city', 'zipcode', 'state', 'longitude', 'latitude', 'host_response_rate', 'host_acceptance_rate', 'host_listings_count'], axis=1)


# Gerer les valeurs manquantes
## colonne ou peu de valeurs sont manquantes
les colonnes 'bderooms', 'bathrooms' et 'beds' contienne peu de valeur manquante. On peut sans problème supprimer les lignes ou ces valeurs sont manquantes, sans perdre beaucoup de datas
## colonne ou beaucoup de valeurs manquent
les colonnes cleaning_fee et security_deposit contiennent beaucoup de valeurs manquantes. Supprimer ces lignes représente une grosse perde de données. Nous allons supprimer ces colonnes

In [37]:
paris_listing.isnull().sum()

accommodates            0
bedrooms               24
bathrooms              58
beds                   14
price                   0
cleaning_fee         1750
security_deposit     1680
minimum_nights          0
maximum_nights          0
number_of_reviews       0
dtype: int64

In [38]:
paris_listing = paris_listing.drop(['cleaning_fee', 'security_deposit'], axis=1)
paris_listing = paris_listing.dropna(axis=0)
paris_listing.isnull().sum()

accommodates         0
bedrooms             0
bathrooms            0
beds                 0
price                0
minimum_nights       0
maximum_nights       0
number_of_reviews    0
dtype: int64

# Normalisation des colonnes
Certaines colonnes du dataset contiennent des chiffres et des écarts très grand (number of review par ex.). C'est colonne peuvent prendre un poids démesurer lors de la prediction. Il est nécéssaire de normaliser les colonnes afin de garantir que chaque colonne ai un poids comparable

Distribution normal standard : formule : 
$\ x=\frac{x-\mu}{\sigma}$

ou $\ x$ est la valeur a normaliser, $\mu$ la moyenne de la colonne normalisé et $\sigma$ la déviation standard (ou écart type)

In [39]:
normalized_listing = (paris_listing - paris_listing.mean()) / paris_listing.std()

In [40]:
normalized_listing['price'] = paris_listing['price']

In [41]:
normalized_listing.head(3)

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
4740,0.503868,-0.296884,-0.293537,0.20531,65.0,-0.184601,1.062858,-0.564545
5606,-0.131849,0.892605,0.843973,0.20531,98.0,-0.101183,1.061018,-0.636924
4824,-0.767566,-1.486372,-0.293537,-0.64526,65.0,0.037847,1.062858,-0.651400


# Distance Euclidienne pour cas multivarié

$\ d=\sqrt{(q_1 - p_1)^2 + (q_2 - p_2)^2 + ... + (q_n - p_n)^2}$

- Calculer la distance Euclidienne en utilisant accommodates & bedrooms entre la premiere et la 5è ligne

In [43]:
from scipy.spatial import distance

list_1 = normalized_listing.iloc[0][['accommodates', 'bedrooms']]
list_2 = normalized_listing.iloc[4][['accommodates', 'bedrooms']]
distance = distance.euclidean(list_1, list_2)
print(distance)

0.6357172321498354
